Source api: [OPEN-METEO](https://open-meteo.com/)

In [1]:
import requests as req
import pandas as pd
import matplotlib.pyplot as plt
import requests_cache
from retry_requests import retry
import openmeteo_requests

In [2]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [3]:
weather_folder_path = './data/weather'

In [4]:
years = [('2017-01-01', '2017-12-31'),
         ('2018-01-01', '2018-12-31'),
         ('2019-01-01', '2019-12-31'),
         ('2020-01-01', '2020-12-31'),
         ('2021-01-01', '2021-12-31'),
         ('2022-01-01', '2022-12-31'),
         ('2023-01-01', '2023-12-31'),
         ('2024-01-01', '2024-03-24')]

In [5]:
features = ['temperature_2m',
           'relative_humidity_2m',
           'pressure_msl',
           'surface_pressure',
           'precipitation',
           'rain',
           'cloud_cover',
           'cloud_cover_low',
           'cloud_cover_mid',
           'cloud_cover_high',
           'shortwave_radiation',
           'direct_radiation',
           'diffuse_radiation',
           'sunshine_duration',
           'wind_speed_10m',
           'wind_speed_100m',
           'wind_direction_10m',
           'wind_direction_100m',
           'wind_gusts_10m',]

In [6]:
bp = {'lat': 47.4925, 'long': 19.051389, 'name' : 'BP'}
konst = {'lat': 44.1811, 'long': 28.5476, 'name' : 'Konstanca'}

In [7]:
url = "https://archive-api.open-meteo.com/v1/archive"

In [8]:
def get_data(location, start_date, end_date, features):
    features_str = ','.join(features)
    features_str

    params = {
        "latitude": location.get('lat'),
        "longitude": location.get('long'),
        "start_date": start_date,
        "end_date": end_date,
        "hourly":  features_str
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    hourly = response.Hourly()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
        )}
    
    for f in features:
        hourly_data[f"{f}_{location.get('name')}"] = hourly.Variables(features.index(f)).ValuesAsNumpy()

    df = pd.DataFrame(hourly_data)
    return df


In [9]:
locations = [bp, konst]

In [10]:
for loc in locations:
    for year in years:
        df = get_data(loc, year[0], year[1], features)
        df_final = df if year == years[0] else pd.concat([df_final, df])
    df_final.to_csv(f'{weather_folder_path}/weather_{loc.get('name')}_{years[0][0],years[-1][-1]}.csv', index = False)

In [11]:
df_final

,date,temperature_2m_Konstanca,relative_humidity_2m_Konstanca,pressure_msl_Konstanca,surface_pressure_Konstanca,precipitation_Konstanca,rain_Konstanca,cloud_cover_Konstanca,cloud_cover_low_Konstanca,cloud_cover_mid_Konstanca,cloud_cover_high_Konstanca,shortwave_radiation_Konstanca,direct_radiation_Konstanca,diffuse_radiation_Konstanca,sunshine_duration_Konstanca,wind_speed_10m_Konstanca,wind_speed_100m_Konstanca,wind_direction_10m_Konstanca,wind_direction_100m_Konstanca,wind_gusts_10m_Konstanca
0,2017-01-01 00:00:00+00:00,-5.684500,85.782043,1025.099976,1017.669006,0.0,0.0,0.900000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,16.055353,31.154043,289.653900,292.416199,27.719999
1,2017-01-01 01:00:00+00:00,-4.734500,82.648636,1024.500000,1017.099670,0.0,0.0,22.800001,0.0,38.0,0.0,0.0,0.0,0.0,0.0,17.317459,31.883888,290.695526,295.407684,29.519999
2,2017-01-01 02:00:00+00:00,-4.734500,81.698486,1023.700012,1016.305420,0.0,0.0,16.800001,0.0,28.0,0.0,0.0,0.0,0.0,0.0,17.992796,32.862694,289.885254,294.599762,30.960001
3,2017-01-01 03:00:00+00:00,-4.634500,80.461502,1023.200012,1015.811829,0.0,0.0,18.000000,0.0,30.0,0.0,0.0,0.0,0.0,0.0,18.899143,33.759480,287.744751,291.914917,32.399998
4,2017-01-01 04:00:00+00:00,-4.684500,79.834877,1023.000000,1015.611694,0.0,0.0,13.200001,0.0,22.0,0.0,0.0,0.0,0.0,0.0,19.211996,33.931789,282.994629,287.281586,32.760002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,2024-03-24 19:00:00+00:00,10.815499,83.978157,1002.500000,995.653564,0.0,0.0,22.200001,0.0,1.0,72.0,0.0,0.0,0.0,0.0,11.304229,26.987997,217.234909,223.919159,20.519999
2012,2024-03-24 20:00:00+00:00,10.465500,83.084564,1001.900024,995.049194,0.0,0.0,51.600002,0.0,36.0,100.0,0.0,0.0,0.0,0.0,10.799999,26.220753,216.869980,224.443817,18.359999
2013,2024-03-24 21:00:00+00:00,10.965500,78.470284,1001.299988,994.465149,0.0,0.0,19.800001,0.0,33.0,0.0,0.0,0.0,0.0,0.0,12.313894,28.284639,217.875046,222.420807,20.160000
2014,2024-03-24 22:00:00+00:00,10.815499,76.852478,1001.000000,994.163757,0.0,0.0,28.500000,0.0,47.0,1.0,0.0,0.0,0.0,0.0,12.496719,28.817993,258.366394,257.005371,20.519999
